<a href="https://colab.research.google.com/github/mohammad-rahbari/Federated-Learning-MLDL/blob/master/notebooks/MLDL_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import pandas
import numpy as np

# Clean up
don't run this section

In [ ]:
df_global = pandas.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")
df_client = pandas.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/client_log_recovery.csv", on_bad_lines='skip')
df_client2 = pandas.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/client_log.csv", on_bad_lines='skip')


df_client = pandas.concat([df_client, df_client2])


In [ ]:
df_client.to_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/client_log.csv", index=False)

In [ ]:
def get_Nc(global_df,client_df):
  result = {}
  for i in range(len(global_df)):
    temp = client_df[client_df["initial_model_name"] == global_df.iloc[i]["model_name"]]
    global_model_name =  global_df.iloc[i]["model_name"]
    if len(temp) == 0:
      classes = None
    else:
      classes = temp["classes"].values[0]

    if classes is None:

      Nc = 0
    elif classes == "all":
      Nc = 100
    else:

      Nc = len(classes.split(","))

    result[global_model_name] = Nc
  return result

Nc_dict = get_Nc(df_global, df_client)

def get_num_local_steps(global_df,client_df):
  result = {}
  for i in range(len(global_df)):
    temp = client_df[client_df["initial_model_name"] == global_df.iloc[i]["model_name"]]
    global_model_name =  global_df.iloc[i]["model_name"]

    j = max(temp["num_of_local_steps"].values)

    result[global_model_name] = j
  return result


df_global["Nc"] = [ None ]* len(df_global)
df_global = df_global[['backbone', 'num_of_clients', 'splitting_method','Nc', 'aggregation_method',
       'Measurement_criteria', 'accuracy', 'loss', 'size_of_dataset',
       'train_test_ratio', 'classes', 'round_number', 'num_of_participants',
       'model_name', 'prev_global_model_name', 'contributors', 'path',
       'momentum_vector_path', 'path_to_subsets', 'path_to_class_combs',
       'time']]

Nc_lst = []
for i in range(len(df_global)):
  Nc_lst.append(Nc_dict[df_global.iloc[i]["model_name"]])

df_global["Nc"] = Nc_lst



In [ ]:


def get_num_local_steps(global_df,client_df):
  result = {}
  for i in range(len(global_df)):
    temp = client_df[client_df["initial_model_name"] == global_df.iloc[i]["model_name"]]

    global_model_name =  global_df.iloc[i]["model_name"]
    try:
      j = max(temp["num_local_steps"].values)
    except:
      j = 0
    result[global_model_name] = j
  return result

num_local_steps_dict = get_num_local_steps(df_global, df_client)

df_global["num_local_steps"] = [ None ]* len(df_global)

num_local_steps_lst = []
for i in range(len(df_global)):
  num_local_steps_lst.append(num_local_steps_dict[df_global.iloc[i]["model_name"]])

df_global["num_local_steps"] = num_local_steps_lst



In [ ]:
# df_global = df_global[['backbone', 'num_of_clients', 'splitting_method','Nc','num_local_steps', 'aggregation_method',
#        'Measurement_criteria', 'accuracy', 'loss', 'size_of_dataset',
#        'train_test_ratio', 'round_number', 'num_of_participants',
#        'model_name', 'prev_global_model_name', 'contributors', 'path',
#        'momentum_vector_path', 'path_to_subsets', 'path_to_class_combs',
#        'time']]
# df_global.to_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv",index=False )

# Extract trained setting

In [35]:
df_global = pandas.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")
df_client = pandas.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/client_log.csv")

def get_seed_models(df):
  filter = df['round_number'] == 0
  df_filtered = df[filter]
  return df_filtered

def get_model_squence(df, seed_model):
  result = df[df['model_name']==seed_model['model_name']]
  latest_model = result['model_name'].values[0]
  while True:
    if latest_model in df['prev_global_model_name'].values:
      temp =  df[df['prev_global_model_name']==latest_model]
      latest_model = temp['model_name'].values[0]
      result = pandas.concat([result, temp])
    else:
      break
  result = result.sort_values(by=['round_number'])
  return result

def get_from_client(df_client, model_name, attribute, single_record= True):
  if single_record:
    return df_client[df_client['initial_model_name']==model_name][attribute].values[0]
  else:
    return df_client[df_client['initial_model_name']==model_name][attribute]

def get_vlid_lst(lst):
  result = []
  for i in lst:

    if not (i is None or np.isnan(i)):
      result.append(i)

  return result

In [44]:
seed_models = get_seed_models(df_global)

for i in range(len(seed_models)):

  log = ""


  df_seq = get_model_squence(df_global,seed_models.iloc[i] )
  r_num = max(get_vlid_lst(df_seq['round_number'].values))
  if r_num != 0:

    log +=f"Seed model: {seed_models.iloc[i]['model_name']} \n"
    log +=f"Spliting method: {df_seq['splitting_method'].values[-1]}  \n"
    if df_seq['splitting_method'].values[-1] == "non-i.i.d. sharing":
      log +=f"Nc: {max(get_vlid_lst(df_seq['Nc'].values))}  \n"
    log +=f"Number of rounds: {r_num}  \n"
    log +=f"Number of local_steps: {max(get_vlid_lst(df_seq['num_local_steps'].values))}  \n"
    try:
      log +=f"Target Sparcity: {get_from_client(df_client, seed_models.iloc[i]['model_name'], 'sparsity')}  \n"
      log +=f"Accuracy: {df_seq[df_seq['round_number']== r_num]['accuracy'].values[-1]}  \n"
      log +=f"Loss: {df_seq[df_seq['round_number']== r_num]['loss'].values[-1]} \n"
    except:
      continue

    print(log," \n \n \n")







Seed model: a3030af8-712b-4617-bbee-ab67d3dff2d7 
Spliting method: i.i.d. sharing  
Number of rounds: 150  
Number of local_steps: 5.0  
Target Sparcity: 0.9  
Accuracy: 28.37  
Loss: 2.907258399581909 
  
 
 

Seed model: f142f936-1ed1-4dfb-b0b2-3f99aff46a89 
Spliting method: i.i.d. sharing  
Number of rounds: 101  
Number of local_steps: 4.0  
Target Sparcity: 0.5  
Accuracy: 37.06  
Loss: 2.498008365631104 
  
 
 

Seed model: 41715a10-15ab-4c46-81ec-18ade92789db 
Spliting method: non-i.i.d. sharing  
Nc: 100.0  
Number of rounds: 2  
Number of local_steps: 4.0  
Target Sparcity: 0.5  
Accuracy: 1.62  
Loss: 6.56297015914917 
  
 
 

Seed model: f56d4e15-9536-4733-9120-6423230a650d 
Spliting method: non-i.i.d. sharing  
Nc: 50.0  
Number of rounds: 101  
Number of local_steps: 4.0  
Target Sparcity: 0.5  
Accuracy: 42.91  
Loss: 2.204365297317505 
  
 
 

Seed model: e7afd632-fee2-4218-b6cc-0636438b2edb 
Spliting method: non-i.i.d. sharing  
Nc: 20.0  
Number of rounds: 37  
Number 